In [ ]:
meta_path = "D:/models/uno/best_model.pt"


res_val_path = "D:/data/metric/all/val/pino"
res_test_path = "D:/data/metric/all/test/pino"

csv_path = "D:/data/metric/all_csv"

In [ ]:
from rtmag.process.paper.utils import load_yaml_to_dict

config_dict = load_yaml_to_dict("metrics.yaml")

dataset_path = config_dict["dataset_path"]
exc_noaa = config_dict["exc_noaa"]
val_path = config_dict["val_path"]
test_path = config_dict["test_path"]

In [ ]:
from pathlib import Path

In [ ]:
files = list(Path(dataset_path).glob('**/label/*.npz'))
if isinstance(exc_noaa, str):
    files = sorted([f for f in files if not exc_noaa == str(f.parent.parent.stem)])
elif isinstance(exc_noaa, list):
    files = sorted([f for f in files if not exc_noaa[0] == str(f.parent.parent.stem)])
    for t_noaa in exc_noaa[1:]:
        files = sorted([f for f in files if not t_noaa == str(f.parent.parent.stem)])

In [ ]:
val_files = []
for d_path in val_path:
    val_files += list(Path(d_path).glob('**/label/*.npz'))
val_files = sorted([f for f in val_files])

In [ ]:
test_files = []
for d_path in test_path:
    test_files += list(Path(d_path).glob('**/label/*.npz'))
test_files = sorted([f for f in test_files])

In [ ]:
len(files), len(val_files), len(test_files)

In [ ]:
import numpy as np
from tqdm import tqdm

from rtmag.process.paper.utils import get_label
from rtmag.process.paper.load import MyModel
device = "cuda"

In [ ]:
mm = MyModel(meta_path, device=device)

In [ ]:
res_val_path = Path(res_val_path)
res_val_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for file in tqdm(val_files):
    target = res_val_path / f"{file.stem}_pino.npy"
    if not target.exists():
        b_isee = get_label(file)
        b_bottom = b_isee[:, :, 0, :]
        b_bottom = b_bottom.transpose((1, 0, 2))[None, None, ...]

        b = mm.get_pred_from_numpy(b_bottom, ret=False)
        np.save(target, b)

In [ ]:
res_test_path = Path(res_test_path)
res_test_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for file in tqdm(test_files):
    target = res_test_path / f"{file.stem}_pino.npy"
    if not target.exists():
        b_isee = get_label(file)
        b_bottom = b_isee[:, :, 0, :]
        b_bottom = b_bottom.transpose((1, 0, 2))[None, None, ...]

        b = mm.get_pred_from_numpy(b_bottom, ret=False)
        np.save(target, b)

In [ ]:
import pandas as pd
import rtmag.process.paper.metric as m

In [ ]:
csv_path = Path(csv_path)
csv_path.mkdir(exist_ok=True, parents=True)

In [ ]:
results = []
for file in tqdm(val_files):
    b_isee = get_label(file)

    target = res_val_path / f"{file.stem}_pino.npy"
    b_pot = np.load(target)

    res = m.evaluate(b_pot, b_isee, ret=False)
    res.update({'file':file.stem})
    results.append(res)

    df = pd.DataFrame.from_dict(results)
    df.to_csv(csv_path / 'metrics_val_pino.csv', index=False)

In [ ]:
results = []
for file in tqdm(test_files):
    b_isee = get_label(file)

    target = res_test_path / f"{file.stem}_pino.npy"
    b_pot = np.load(target)

    res = m.evaluate(b_pot, b_isee, ret=False)
    res.update({'file':file.stem})
    results.append(res)

    df = pd.DataFrame.from_dict(results)
    df.to_csv(csv_path / 'metrics_test_pino.csv', index=False)